<a href="https://colab.research.google.com/github/miladmirzazadeh/Gamein-Prototype/blob/master/prototype_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



TESTING CODES ############################################################################



In [ ]:
df = pd.DataFrame([[0, 2, 3], [0, 4, 1], [10, 20, 30]],
                  index=[4, 5, 6], columns=['A', 'B', 'C'])

df

,A,B,C
4,0,2,3
5,0,4,1
6,10,20,30


In [ ]:
for i,row in df.iterrows():
  for ind,ro in row.iteritems() :
    print(ro)
  break

0
2
3


In [16]:
Controller.create_player("raman")
Controller.create_player("ali") 

raman = Controller.player_objects["raman"]
raman_stockhouse = list(raman.land_objects.values())[0].stockhouse

ali = Controller.player_objects["ali"]
ali_stockhouse = list(ali.land_objects.values())[0].stockhouse


In [17]:
Controller.start_production("ali","چوب", 1110)

ValueError: ignored

In [ ]:
ali_stockhouse.inventory_df

,total_inventory,inhand_inventory
tomato,1110,1110


In [ ]:
Controller.put_into_market("ali", "tomato", 500, 200)

'Done :)'

In [ ]:
Controller.show_product_market_info()

,id,product_name,amount,price,seller_name
88,88,tomato,400,200,ali
29,29,tomato,500,200,ali


In [ ]:
Controller.buy_product("ali",Market.vitrin.index[0],100)

'Done :)'

In [ ]:
ali.cash

1000000

,id,product_name,amount,price,seller_name
40,40,potato,400,200,raman
506,506,tomato,500,200,ali


END   TESTING CODES ######################################################################









In [2]:
import numpy as np 
import pandas as pd 
import random

In [28]:
type(pd.read_excel("/content/drive/My Drive/gamein_prototype/parameters.xlsx",sheet_name="land_machine", header=0,index_col=0).loc[30001])

pandas.core.series.Series

In [29]:
class Controller():

  # We have to set All initial values here ...... why ? because we wanna reset them everytime we build Controller class 
  Market.vitrin = pd.DataFrame([], columns=["seller_obj","product_name","amount", "price"])
  Land.available_lands = Land.lands_list
  ##############################
  return_message=""
  turn = 0 
  last_turn = 50
  player_objects = dict()#name,object

  def next_turn():
    turn+=1
    if turn == Controller.last_turn : 
      score_board = Controller.score_board()
      Controller.return_message = "End Game. Thank u " + score_board

  
  def score_board():
    net_worths = pd.Series([])
    for player in list(player_objects.keys()):
      obj = player_objects[player]
      obj.calculate_final_net_worth()
      net_worths[player] = obj 
    net_worths = net_worths.sort(ascending=False)
    return net_worths


  # def get_status(player_name):
    # for land in player_objecs[player_name].

  def create_player(player_name):
    land_id = random.choice(Land.available_lands) #random از بین لیست زمین های در دسترس
    Land.available_lands.remove(land_id)   #remove from available lands
    
    machine_id =  Parameters.land_machine_df.loc[land_id, machine_id] # از دیتافریم زمین و ماشین خوانده میشود
    
    Controller.player_objects.update({player_name : Player(player_name)})
    Controller.buy_land(player_name, land_id)
    Controller.buy_machine(player_name, land_id, machine_id)
    
  def buy_land(player_name, land_id):
    Controller.player_objects[player_name].add_land(land_id)
    # Controller.player_objects[name].cash -=         اینجا باید پول زمین رو از حسابش کم کنیم . فعلا لازم نیست  چون خودمون بهش زمین رو اختصاص میدیم 

  def buy_machine(player_name, land_id, machine_id):
    Controller.player_objects[player_name].add_machine(land_id, machine_id)

  def start_production(player_name, product_name, amount):# This Func should take machine_id too ( Later Inshallah)
    player_obj = Controller.player_objects[player_name]
    stockhouse = list(player_obj.land_objects.values())[0].stockhouse
    if not (product_name in Product.products_df.index): ### checking existence
      return ("Do we have this Product in the Game ?!") 
    product_inf = Product.products_df.loc[product_name]
    cost = product_inf.production_cost * amount
    volume = _ * amount # _ is Defined

    if player_obj.cash <= cost :  ############## Checking Player Cash
      return ("No Enough Cash")
    if stockhouse.remaining_cap <= volume: ############### Checking Inventory Remaining Cap
      return ("No Enough Space")
  
    materials = product_inf.drop(["production_cost", "lower_band", "upper_band"]) ############ Checking Materials
    for mat, number in materials.iteritems():
      if number > 0 :
        if (not (mat in stockhouse.inventory_df.index) or (number*amount > stockhouse.inventory_df.loc[mat])):
          return ("No enough" + mat)
        stockhouse.remove(mat, number*amount, just_total = False)
    ##################################### when u get here , everything is OK
    player_obj.cash -= cost
    print("yesss")
    stockhouse.add(product_name, amount)
    
      


  def put_into_market(player_name, product_name, amount, price):# it doesn't get stockhouse_id or etc. until we have more than one stockhouse
    player_obj = Controller.player_objects[player_name]
    product_on_vitrin_id = random.choice([i for i in range(100) if not(i in Market.vitrin.index)])   # a random unique id , that will assign to vitrin objects
    stockhouse_obj = list(player_obj.land_objects.values())[0].stockhouse
    

    #check the inventory
    if not (product_name in stockhouse_obj.inventory_df.index):
      return ("Doesn't Exist")

    if stockhouse_obj.inventory_df.loc[product_name, "inhand_inventory"] < amount : 
      return ("No Enough Inventory")
    
    existing_row = Market.vitrin[np.logical_and(np.logical_and(Market.vitrin.product_name == product_name, Market.vitrin.amount== amoun), Market.vitrin.seller_obj == player_obj)]
    if len(existing_row) != 0:
      # add to existing row IF player_name, product_name and price are equal 
      Market.vitrin.loc[existing_row.index, "amount"] += amount
    else : 
      Market.vitrin.loc[product_on_vitrin_id] = [player_obj, product_name, amount, price]
      stockhouse_obj.inventory_df.loc[product_name, "inhand_inventory"] -= amount
    return ("Done :)")
    
  
  def upgrade_stockhouse(player_name, land_id):
    pass

  def show_product_market_info():
    vitrin = Market.vitrin.copy()
    vitrin["seller_name"] = [obj.name for obj in vitrin.seller_obj]
    vitrin["id"] = vitrin.index
    vitrin = vitrin[["id", "product_name" , "amount", "price", "seller_name"]]
    return vitrin

  def buy_product(player_name, product_on_vitrin_id, amount): # آیدی محصول داخل ویترین رو میده که بریم از روی دیکشنری توی مارکت آبجکتش رو پیدا کنیم و انتقال رو انجام بدیم 
    buyer_player = Controller.player_objects[player_name]
    buyer_stockhouse = list(buyer_player.land_objects.values())[0].stockhouse
    product_on_vitrin_obj = Market.vitrin.loc[product_on_vitrin_id]
    seller_player = product_on_vitrin_obj.seller_obj
    
    if amount > buyer_stockhouse.remaining_cap:
      return ("No Enough Space")
    elif buyer_player.cash < product_on_vitrin_obj.price * amount :
      return ("No Enough Cash")
    elif amount > product_on_vitrin_obj.amount :
      return ("No Enough Product In Vitrin")
    else:
      buyer_player.cash -= amount * product_on_vitrin_obj.price #update cash
      buyer_stockhouse.add(product_on_vitrin_obj.product_name, amount) #updata buuyer inventory


      #update seller inventory :
      seller_stockhouse = list(seller_player.land_objects.values())[0].stockhouse
      seller_stockhouse.remove(product_on_vitrin_obj.product_name, amount)
      seller_player.cash += amount * product_on_vitrin_obj.price # add cash to seller player


      #update vitrin
      product_on_vitrin_obj.amount -= amount
      if product_on_vitrin_obj.amount == 0 : #### deleting vitrin row ir amount == 0 
        Market.vitrin.drop(product_on_vitrin_id)
        

      return ("Done :)")
    
   

In [4]:
class Player():
  players_name=[]

  def __init__(self, player_name):
    self.name = player_name
    #remove from available
    self.cash = 1000000 # بودجه اولیه
    self.land_objects=dict()

  def calculate_final_net_worth(self):
    inventory_value_list = []
    for land in list(self.land_objects.keys()):
      inventory_value_list.append(self.land_objects[land].calculate_inventory_value()) # تو هر خونه این لیست ، ارزش دارایی های یک زمین از زمین های بازیکن ثبت میشه
    inventory_value_sum = sum(inventory_value_list)
    net_worth = inventory_value_sum + self.cash 
    return net_worth 

  def add_land(self, land_id):
    self.land_objects.update({land_id : Land(land_id)}) # یه اینستنس از لند میسازه که ورودی بهش آیدی میدیم . بعد توی خود لند ، بر حسب آیدی بقیه فیچر های لند رو ست میکنیم
    

  def add_machine(self, land_id, machine_id):
    self.land_objects[land_id].add_machine(machine_id)







In [5]:
class Gamein_Bazaar():
  def put_into_market():
    pass

In [6]:
class Land():
  lands_list = [30001,30002,30003,30004,30005,30006,30007,30008,
                30009,300010,300011,300012,300013,300014,300015] #amirreza یه دیتا فریم که یه آی دی زمین داره و ماشین
  available_lands= lands_list#آی دی های باقی مانده
  
  def __init__(self, land_id):
    
    self.cap = Parameters.land_stockhouse
    self.machine_objects = dict()
    self.land_id = land_id
    self.stockhouse = StockHouse(cap)# اینجا دیگه دیکشنری نیست چون فرض بر اینه که یه دونه انبار بیشتر نداریم 


    # self.stockhouse = StockHouse()
  def add_machine(self, machine_id):
    self.machine_objects.update({machine_id: Machine(machine_id)})

    


  

In [7]:

class StockHouse():
  def __init__(self, cap):
    self.total_cap = cap
    self.remaining_cap = self.total_cap
    self.inventory_df = pd.DataFrame([], columns= ["total_inventory", "inhand_inventory"]) # indexes = product names 
    self.cost_per_product = 100
  def add(self, product_name, amount):
    if self.remaining_cap <= amount : 
      return ("No Enough Space")
    else :
      if product_name in self.inventory_df.index:
        self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] += amount
      else:
        self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] = amount 
      self.remaining_cap = self.total_cap - self.inventory_df.total_inventory.sum()
      return (True)
  def remove(self, product_name, amount, just_total=True):
    volume = amount * _ # _ From Csv  
    self.inventory_df.loc[product_name, ["total_inventory"]] -= amount 
    self.remaining_cap += volumne 
    if not just_total : 
      self.inventory_df.loc[product_name, ["inhand_inventory"]] -= amount


    



  

In [24]:
class Machine():


  def __init__(self,machine_id):
    self.product_name = _
    # self.price = _ 
    self.enabled = True
    self.production_cost_per_unit = _
    self.fixed_production_cost = _
    self.working = 0
    self.remaining_production = _
    
  def start_production(self):
    pass
  def stop_production(self):
    pass
  

  

In [21]:
pd.read_excel("/content/drive/My Drive/gamein_prototype/parameters.xlsx",sheet_name="bom", header=0,index_col=0)

,الوار,چوب,نخ,پارچه,رنگ,آهن,میخ
چوب,1,0,0,0,0,0,0
پارچه,0,0,500,0,0,0,0
میخ,0,0,0,0,0,20,0
کمد,0,2,0,0,2,0,50
صندلی,0,1,0,1,1,0,0


In [10]:
class Product():

  products_bom = pd.read_excel("/content/drive/My Drive/gamein_prototype/parameters.xlsx",sheet_name="bom", header=0,index_col=0)
  products_space = _
  
  def __init__(self):

    self.material_needed = 10
    self.lower_band = 1
    self.upper_band = 20

    
  

In [11]:
Product.products_df

,production_cost,lower_band,upper_band,الوار,چوب,نخ,پارچه,رنگ,آهن,میخ
چوب,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
پارچه,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
میخ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
کمد,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
صندلی,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
class Material():
  Prices = _ #dict (id , price)

In [13]:
class Market():
  vitrin = pd.DataFrame([], columns=["seller_obj","product_name", "amount", "price"])


In [14]:
#####################################################
#################################################### DELETED


class ProductOnVitrin():
  def __init__(self, ):
    self.seller_obj = player_obj
    self.stockhouse_obj = stockhouse_obj
    self.product_obj = product_obj
    self.amount = amount
    self.price = price

In [31]:
class Parameters():
  ###################### reading all the dataframes from google drive ...... to call them faster and easier 

  land_stockhouse_df = pd.read_excel("/content/drive/My Drive/gamein_prototype/parameters.xlsx",sheet_name="land_stockhouse", header=0,index_col=0)
  land_machine_df = pd.read_excel("/content/drive/My Drive/gamein_prototype/parameters.xlsx",sheet_name="land_machine", header=0,index_col=0)
  machines_info_df = pd.read_excel("/content/drive/My Drive/gamein_prototype/parameters.xlsx",sheet_name="machines_info", header=0,index_col=0)
  bom_df = pd.read_excel("/content/drive/My Drive/gamein_prototype/parameters.xlsx",sheet_name="bom", header=0,index_col=0)